### Repositorio
https://github.com/Diego2436/Inteligencia-Artificial/
### Nombres
- Olmos Verdin Diego
- Romero Hernández Oscar David
- Valentin Ramos Emmanuel Guadalupe

In [1]:
import numpy as np
from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, KFold, LeaveOneOut
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd

C:\Users\diego\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Se importa `numpy` para operaciones matemáticas y `pandas` para la manipulación de datos. `scikit-learn` se utiliza para cargar conjuntos de datos de ejemplo (`load_iris`, `load_wine`, `load_breast_cancer`), así como para realizar particiones de datos (`train_test_split`) y validación cruzada (`cross_val_score`, `KFold`, `LeaveOneOut`). Además, se importan clasificadores como `GaussianNB` (Naive Bayes) y `KNeighborsClassifier` (k-NN), junto con métricas de evaluación como `accuracy_score` y `confusion_matrix` para calcular la precisión y la matriz de confusión de las predicciones. El código proporciona las herramientas necesarias para aplicar diferentes algoritmos de clasificación y evaluar su desempeño en los conjuntos de datos de ejemplo utilizando validación cruzada y otros métodos de partición de datos.

In [2]:
# Cargar los datasets
datasets = {
    "Iris": load_iris(),
    "Wine": load_wine(),
    "Breast Cancer": load_breast_cancer()
}

# Inicializar los clasificadores
nb_classifier = GaussianNB()
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # Valor de K puede ajustarse según conveniencia

Se carga tres conjuntos de datos de ejemplo (`Iris`, `Wine`, y `Breast Cancer`) utilizando las funciones `load_iris()`, `load_wine()`, y `load_breast_cancer()` de la biblioteca `scikit-learn`, y los almacena en un diccionario llamado `datasets`, donde las claves son los nombres de los datasets y los valores son los datos cargados. Además, inicializa dos clasificadores: `nb_classifier`, que es un clasificador Naive Bayes (`GaussianNB()`), y `knn_classifier`, que es un clasificador k-Vecino más Cercano (`KNeighborsClassifier`), con `k=5` (el número de vecinos puede ajustarse según la necesidad del usuario). Estos clasificadores se utilizarán más adelante para realizar predicciones sobre los datos de los conjuntos mencionados.

In [3]:
# Función para entrenar y evaluar un clasificador
def evaluate_classifier(name, classifier, X, y):
    results = {"Method": [], "Accuracy": [], "Confusion Matrix": []}

    # Hold-Out 70/30
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    results["Method"].append("Hold-Out 70/30")
    results["Accuracy"].append(accuracy)
    results["Confusion Matrix"].append(cm)

    # 10-Fold Cross-Validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    cv_scores = cross_val_score(classifier, X, y, cv=kf)
    accuracy = cv_scores.mean()
    classifier.fit(X, y)
    y_pred = classifier.predict(X)
    cm = confusion_matrix(y, y_pred)
    results["Method"].append("10-Fold Cross-Validation")
    results["Accuracy"].append(accuracy)
    results["Confusion Matrix"].append(cm)

    # Leave-One-Out Cross-Validation
    loo = LeaveOneOut()
    loo_scores = cross_val_score(classifier, X, y, cv=loo)
    accuracy = loo_scores.mean()
    classifier.fit(X, y)
    y_pred = classifier.predict(X)
    cm = confusion_matrix(y, y_pred)
    results["Method"].append("Leave-One-Out")
    results["Accuracy"].append(accuracy)
    results["Confusion Matrix"].append(cm)

    return pd.DataFrame(results)

Este código define la función `evaluate_classifier`, que entrena y evalúa un clasificador utilizando tres métodos de validación: Hold-Out 70/30, validación cruzada 10-Fold y validación Leave-One-Out. La función toma como parámetros el nombre del clasificador (`name`), el clasificador en sí (`classifier`), y los datos (`X` para las características y `y` para las etiquetas). Primero, realiza una partición de los datos en entrenamiento y prueba utilizando Hold-Out 70/30, entrena el clasificador en los datos de entrenamiento, realiza predicciones sobre los datos de prueba, calcula la exactitud y la matriz de confusión, y almacena los resultados. Luego, realiza validación cruzada de 10 pliegues (`KFold`), calculando la exactitud promedio de las predicciones y la matriz de confusión sobre todo el conjunto de datos. Finalmente, aplica validación Leave-One-Out (`LeaveOneOut`), repitiendo el proceso para cada muestra del conjunto de datos, y guarda los resultados correspondientes. Los resultados de cada método se almacenan en un diccionario que se convierte en un DataFrame de `pandas`, que incluye el nombre del método, la exactitud obtenida y la matriz de confusión asociada para cada evaluación. Esta función es útil para comparar el rendimiento de un clasificador utilizando distintos métodos de validación.

In [4]:
# Evaluar cada dataset y clasificador
for dataset_name, dataset in datasets.items():
    X, y = dataset.data, dataset.target
    print(f"--- Dataset: {dataset_name} ---\n")

    # Naive Bayes
    print("Naive Bayes Results:")
    nb_results = evaluate_classifier("Naive Bayes", nb_classifier, X, y)
    print(nb_results, "\n")

    # KNN
    print("KNN Results:")
    knn_results = evaluate_classifier("KNN", knn_classifier, X, y)
    print(knn_results, "\n")

--- Dataset: Iris ---

Naive Bayes Results:
                     Method  Accuracy                      Confusion Matrix
0            Hold-Out 70/30  0.977778  [[19, 0, 0], [0, 12, 1], [0, 0, 13]]
1  10-Fold Cross-Validation  0.960000  [[50, 0, 0], [0, 47, 3], [0, 3, 47]]
2             Leave-One-Out  0.953333  [[50, 0, 0], [0, 47, 3], [0, 3, 47]] 

KNN Results:
                     Method  Accuracy                      Confusion Matrix
0            Hold-Out 70/30  1.000000  [[19, 0, 0], [0, 13, 0], [0, 0, 13]]
1  10-Fold Cross-Validation  0.973333  [[50, 0, 0], [0, 47, 3], [0, 2, 48]]
2             Leave-One-Out  0.966667  [[50, 0, 0], [0, 47, 3], [0, 2, 48]] 

--- Dataset: Wine ---

Naive Bayes Results:
                     Method  Accuracy                      Confusion Matrix
0            Hold-Out 70/30  1.000000  [[19, 0, 0], [0, 21, 0], [0, 0, 14]]
1  10-Fold Cross-Validation  0.977778  [[58, 1, 0], [0, 70, 1], [0, 0, 48]]
2             Leave-One-Out  0.977528  [[58, 1, 0], [0, 70,

Se recorre los conjuntos de datos cargados en el diccionario `datasets` y evalúa dos clasificadores (`Naive Bayes` y `KNN`) utilizando la función `evaluate_classifier`. Para cada conjunto de datos, se extraen las características (`X`) y las etiquetas (`y`), y se imprime el nombre del conjunto de datos. Luego, se realiza una evaluación para el clasificador Naive Bayes, pasando el clasificador `nb_classifier` y los datos a la función `evaluate_classifier`, cuyos resultados (precisión y matriz de confusión para cada método de validación) se imprimen en la consola. El mismo proceso se repite para el clasificador KNN, utilizando `knn_classifier`. Los resultados para ambos clasificadores se muestran por separado para cada conjunto de datos, permitiendo una comparación directa entre el rendimiento de ambos clasificadores en los diferentes métodos de validación (Hold-Out, 10-Fold Cross-Validation y Leave-One-Out).